In [2]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

# Memory hierarchy

## The memory hierarchy of my computer

In [2]:
# Hung-Wei's Desktop
! lscpu | grep "Model name"
! getconf -a | grep CACHE

Model name:                           13th Gen Intel(R) Core(TM) i7-13700
LEVEL1_ICACHE_SIZE                 65536
LEVEL1_ICACHE_ASSOC                
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 32768
LEVEL1_DCACHE_ASSOC                8
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  4194304
LEVEL2_CACHE_ASSOC                 16
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  31457280
LEVEL3_CACHE_ASSOC                 12
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  
LEVEL4_CACHE_ASSOC                 
LEVEL4_CACHE_LINESIZE              


In [3]:
# Hung-Wei's Desktop
! ssh articuno "lscpu | grep 'Model name'; getconf -a | grep CACHE"

Model name:                           Intel(R) Core(TM) i7-14700K
LEVEL1_ICACHE_SIZE                 65536
LEVEL1_ICACHE_ASSOC                
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 32768
LEVEL1_DCACHE_ASSOC                8
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  4194304
LEVEL2_CACHE_ASSOC                 16
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  34603008
LEVEL3_CACHE_ASSOC                 11
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  
LEVEL4_CACHE_ASSOC                 
LEVEL4_CACHE_LINESIZE              


In [4]:
# Your CS203 Cluster
! cs203 run "lscpu | grep 'Model name'; getconf -a | grep CACHE"

bash: line 0: cd: /nfshome/htseng/courses/CS203/demo/memory: No such file or directory
Model name:                           12th Gen Intel(R) Core(TM) i3-12100F
bash: line 0: cd: /nfshome/htseng/courses/CS203/demo/memory: No such file or directory
LEVEL1_ICACHE_SIZE                 32768
LEVEL1_ICACHE_ASSOC                8
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 49152
LEVEL1_DCACHE_ASSOC                12
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  1310720
LEVEL2_CACHE_ASSOC                 10
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  12582912
LEVEL3_CACHE_ASSOC                 12
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


## How blocks are stored in a cache?

How these address are stored "if" they're in a direct-mapped, 16B-sized blocks, 16-block cache?

In [22]:
!echo "element,address"> addresses.csv; ./mv/matvec 16 1 >> addresses.csv
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
N = 51
df = df.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
index_bits = int(math.log2(number_of_blocks))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

,element,address,tag,index
0,&a[0][0],0x59b848dab330,0x59b848dab3,0x3
1,&b[0],0x59b848dabc30,0x59b848dabc,0x3
2,&a[0][1],0x59b848dab338,0x59b848dab3,0x3
3,&b[1],0x59b848dabc38,0x59b848dabc,0x3
4,&a[0][2],0x59b848dab340,0x59b848dab3,0x4
5,&b[2],0x59b848dabc40,0x59b848dabc,0x4
6,&a[0][3],0x59b848dab348,0x59b848dab3,0x4
7,&b[3],0x59b848dabc48,0x59b848dabc,0x4
8,&a[0][4],0x59b848dab350,0x59b848dab3,0x5
9,&b[4],0x59b848dabc50,0x59b848dabc,0x5


What if we have a 2-way, 16-byte blocked, 16-block cache?

In [6]:
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
N = 51
df = df.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
way_assoc=2
number_of_sets = number_of_blocks/way_assoc
index_bits = int(math.log2(number_of_sets))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

,element,address,tag,index
0,&a[0][0],0x5db036bdc330,0xbb606d7b86,0x3
1,&b[0],0x5db036bdcc30,0xbb606d7b98,0x3
2,&a[0][1],0x5db036bdc338,0xbb606d7b86,0x3
3,&b[1],0x5db036bdcc38,0xbb606d7b98,0x3
4,&a[0][2],0x5db036bdc340,0xbb606d7b86,0x4
5,&b[2],0x5db036bdcc40,0xbb606d7b98,0x4
6,&a[0][3],0x5db036bdc348,0xbb606d7b86,0x4
7,&b[3],0x5db036bdcc48,0xbb606d7b98,0x4
8,&a[0][4],0x5db036bdc350,0xbb606d7b86,0x5
9,&b[4],0x5db036bdcc50,0xbb606d7b98,0x5


## Cache performance of code on "real machines"

### NVIDIA Jetson Nano -- Tegra X1

In [7]:
render_code("4way_madd/madd.c", show=["//START","//END"])

// 4way_madd/madd.c:55-67 (13 lines)
    //START
    for(i = 0; i < RUN_ARRAY_SIZE; i++)
    {
        e[i] = (a[i] * b[i] + c[i])/d[i];
#ifdef DUMP
        fprintf(stderr, "a[%d], %p\n",i,&a[i]);
        fprintf(stderr, "b[%d], %p\n",i,&b[i]);
        fprintf(stderr, "c[%d], %p\n",i,&c[i]);
        fprintf(stderr, "d[%d], %p\n",i,&d[i]);
        fprintf(stderr, "e[%d], %p\n",i,&e[i]);
#endif
    }
  //END

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

#### Run without the 4-way matrix add loop code.

In [8]:
# Run it "without" the above code.
! ssh htseng@nano-2 "lscpu; cd courses/CS203/demo/memory/4way_madd/; make clean madd_nano; valgrind --tool=cachegrind ./madd_nano 16384 0 "

Architecture:        aarch64
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  1
Core(s) per socket:  4
Socket(s):           1
Vendor ID:           ARM
Model:               1
Model name:          Cortex-A57
Stepping:            r1p1
CPU max MHz:         1479.0000
CPU min MHz:         102.0000
BogoMIPS:            38.40
L1d cache:           32K
L1i cache:           48K
L2 cache:            2048K
Flags:               fp asimd evtstrm aes pmull sha1 sha2 crc32
rm -f madd_intel madd_nano *_O3 *~ madd_A_fission cachegrind* *.perf madd_dump
cc -O1 -DHAVE_LINUX_PERF_EVENT_H -g  -DNANO perfstats.c madd.c -o madd_nano
==3546== Cachegrind, a cache and branch-prediction profiler
==3546== Copyright (C) 2002-2017, and GNU GPL'd, by Nicholas Nethercote et al.
==3546== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==3546== Command: ./madd_nano 16384 0
==3546== 
--3546-- Warning: Cannot auto-detect cache config, using defaults.

Too much detail! Let's use grep to narrow down the outputs.

In [9]:
# Run it "without" the above code.
! ssh htseng@nano-2 "cd courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_nano 16384 0 >& nano_without_loop.perf; grep 'D   refs\|D1' nano_without_loop.perf"

==3585== D   refs:      3,009,918  (1,847,746 rd   + 1,162,172 wr)
==3585== D1  misses:       13,790  (    2,863 rd   +    10,927 wr)
==3585== D1  miss rate:       0.5% (      0.2%     +       0.9%  )


Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [10]:
! ssh htseng@nano-2 "cd courses/CS203/demo/memory/4way_madd/;valgrind --tool=cachegrind ./madd_nano 16384 8192 >& nano_with_loop.perf; grep 'D   refs\|D1' nano_with_loop.perf"

==3625== D   refs:      3,050,956  (1,880,570 rd   + 1,170,386 wr)
==3625== D1  misses:       54,768  (   35,647 rd   +    19,121 wr)
==3625== D1  miss rate:       1.8% (      1.9%     +       1.6%  )


In [11]:
# Let's do some math here
total_number_of_accesses_before_the_loop =     3009912
total_number_of_accesses_after_the_loop =   3030476 
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop =  13794
total_number_of_misses_after_the_loop = 34293
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

access in the loop: 20564 misses in the loop 20499 miss_rate 0.996839


In [12]:
! cd 4way_madd; make madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 51 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
N = 32
df = df.iloc[:N]
C = 32768
B = 64
A = 4
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits)%S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

cc -O3 -DHAVE_LINUX_PERF_EVENT_H -g  -DDUMP perfstats.c madd.c -o madd_dump
191600984,77634843,0.405190,0.252734,0.019621,0.000132,10862,82104349


,element,address,tag,index
0,a[0],0x7db975821000,0x3edcbac10,0x40
1,b[0],0x7db975831000,0x3edcbac18,0x40
2,c[0],0x7db975841000,0x3edcbac20,0x40
3,d[0],0x7db975851000,0x3edcbac28,0x40
4,e[0],0x7db975861000,0x3edcbac30,0x40
5,a[1],0x7db975821008,0x3edcbac10,0x40
6,b[1],0x7db975831008,0x3edcbac18,0x40
7,c[1],0x7db975841008,0x3edcbac20,0x40
8,d[1],0x7db975851008,0x3edcbac28,0x40
9,e[1],0x7db975861008,0x3edcbac30,0x40


### Intel Core i7 13700 -- 12-way L1, 64B-blocked, 48KB cache

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

Let's again dump, parse and simulation the address sequence.

In [3]:
! cd 4way_madd; make clean; make EXTRA_CFLAGS="-mno-avx -O4" madd_dump; make EXTRA_CFLAGS="-mno-avx -O4" madd_intel; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 16384 16384 2>> addresses_madd.csv
! head -n 50 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
N = 51
df = df.iloc[:N]
C = 49152
B = 64
A = 12
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

rm -f madd_intel madd_nano *_O3 *~ madd_A_fission cachegrind* *.perf madd_dump
cc -O3 -mno-avx -O4 -DDUMP perfstats.c madd.c -o madd_dump
cc -O3 -mno-avx -O4 -mno-avx perfstats.c madd.c -o madd_intel


,element,address,tag,index
0,a[0],0x79fe0d072000,0x79fe0d072,0x0
1,b[0],0x79fe0d092000,0x79fe0d092,0x0
2,c[0],0x79fe0d0b2000,0x79fe0d0b2,0x0
3,d[0],0x79fe0d0d2000,0x79fe0d0d2,0x0
4,e[0],0x79fe0d0f2000,0x79fe0d0f2,0x0
5,a[1],0x79fe0d072008,0x79fe0d072,0x0
6,b[1],0x79fe0d092008,0x79fe0d092,0x0
7,c[1],0x79fe0d0b2008,0x79fe0d0b2,0x0
8,d[1],0x79fe0d0d2008,0x79fe0d0d2,0x0
9,e[1],0x79fe0d0f2008,0x79fe0d0f2,0x0


#### Run without the 4-way matrix add loop code.

In [4]:
# Run it "without" the above code.
! lscpu; cd ~/courses/CS203/demo/memory/4way_madd/; rm madd_intel; make EXTRA_CFLAGS="-mno-avx -O4" madd_intel; valgrind --tool=cachegrind ./madd_intel 16384 0 >& intel_without_loop.perf; grep 'D   refs\|D1' intel_without_loop.perf

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   24
  On-line CPU(s) list:    0-23
Vendor ID:                GenuineIntel
  Model name:             13th Gen Intel(R) Core(TM) i7-13700
    CPU family:           6
    Model:                183
    Thread(s) per core:   2
    Core(s) per socket:   16
    Socket(s):            1
    Stepping:             1
    CPU(s) scaling MHz:   19%
    CPU max MHz:          5200.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4224.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc c

Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [15]:
# Run it "with" the above code.
! cd ~/courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_intel 16384 16384  >& intel_with_loop.perf; grep 'D   refs\|D1' intel_with_loop.perf

In [16]:
# Let's do some math here
total_number_of_accesses_before_the_loop = 2286839
total_number_of_accesses_after_the_loop = 2327805
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 13458
total_number_of_misses_after_the_loop = 23705
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

access in the loop: 40966 misses in the loop 10247 miss_rate 0.250134


Let's use performance counters to examine the cache/code performance

#### Run with the 4-way matrix add loop code.

In [10]:
! make -C 4way_madd clean madd_intel
! echo "architecture,IC,Cycles,CPI,CycleTime,ET,DL1_Miss_Rate,DL1_MISSES,DL1_ACCESSES" > ./4way_madd/4_way_loop.csv
! echo -n "intel," >> ./4way_madd/4_way_loop.csv
! cd ~/courses/CS203/demo/memory/4way_madd/;./madd_intel 32768 32768 >> 4_way_loop.csv
display_df_mono(render_csv("./4way_madd/4_way_loop.csv"))

make: Entering directory '/nfshome/htseng/courses/CS203/demo/memory/4way_madd'
rm -f madd_intel madd_nano *_O3 *~ madd_A_fission cachegrind* *.perf madd_dump
cc -O3 -DHAVE_LINUX_PERF_EVENT_H -g  -mno-avx perfstats.c madd.c -o madd_intel
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/memory/4way_madd'


,index,architecture,IC,Cycles,CPI,CycleTime,ET,DL1_Miss_Rate,DL1_MISSES,DL1_ACCESSES
0,0,intel,192789,109180,0.566319,0.659461,0.000072,0.120979,10503,86817


In [11]:
! echo -n "nano," >> ./4way_madd/4_way_loop.csv
! ssh htseng@nano-2 "make -C courses/CS203/demo/memory/4way_madd madd_nano"
! ssh htseng@nano-2 "cd courses/CS203/demo/memory/4way_madd/;./madd_nano 32768 32768" >> ./4way_madd/4_way_loop.csv
display_df_mono(render_csv("./4way_madd/4_way_loop.csv"))

make: Entering directory '/nfshome/htseng/courses/CS203/demo/memory/4way_madd'
cc -O1 -DHAVE_LINUX_PERF_EVENT_H -g  -DNANO perfstats.c madd.c -o madd_nano
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/memory/4way_madd'


,index,architecture,IC,Cycles,CPI,CycleTime,ET,DL1_Miss_Rate,DL1_MISSES,DL1_ACCESSES
0,0,intel,192789,109180,0.566319,0.659461,0.000072,0.120979,10503,86817
1,1,nano,378795,957524,2.527816,1.369156,0.001311,0.119231,42040,352594


In [25]:
! make -C 4way_madd clean madd_intel
! echo -n "intel," >> ./4way_madd/4_way_loop.csv
! cd ~/courses/CS203/demo/memory/4way_madd/;./madd_intel 65536 65536 >> 4_way_loop.csv
display_df_mono(render_csv("./4way_madd/4_way_loop.csv"))

make: Entering directory '/nfshome/htseng/courses/CS203/demo/memory/4way_madd'
rm -f madd_intel madd_nano *_O3 *~ madd_A_fission cachegrind* *.perf madd_dump
cc -O3 -DHAVE_LINUX_PERF_EVENT_H -g  -mno-avx perfstats.c madd.c -o madd_intel
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/memory/4way_madd'


,index,architecture,IC,Cycles,CPI,CycleTime,ET,DL1_Miss_Rate,DL1_MISSES,DL1_ACCESSES
0,0,intel,192773,119390,0.619329,0.628193,0.000075,0.127813,11096,86814
1,1,nano,378820,946526,2.498617,1.158975,0.001097,0.119869,41235,344000
2,2,intel,0,0,nan,inf,0.000636,nan,0,0
3,3,intel,375031,182010,0.485320,0.373606,0.000068,0.195028,41383,212190
4,4,nano,739297,1951258,2.639343,0.940931,0.001836,0.110242,74751,678064
5,5,intel,0,0,nan,inf,0.000405,nan,0,0
6,6,nano,739284,1920094,2.597235,1.066094,0.002047,0.248777,176808,710710
7,7,intel,373017,205104,0.549852,0.716710,0.000147,0.128218,21636,168744


In [24]:
! echo -n "nano," >> ./4way_madd/4_way_loop.csv
! ssh htseng@nano-2 "make -C courses/CS203/demo/memory/4way_madd madd_nano"
! ssh htseng@nano-2 "cd courses/CS203/demo/memory/4way_madd/;./madd_nano 65536 65536" >> ./4way_madd/4_way_loop.csv
display_df_mono(render_csv("./4way_madd/4_way_loop.csv"))

make: Entering directory '/nfshome/htseng/courses/CS203/demo/memory/4way_madd'
cc -O1 -DHAVE_LINUX_PERF_EVENT_H -g  -DNANO perfstats.c madd.c -o madd_nano
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/memory/4way_madd'


,index,architecture,IC,Cycles,CPI,CycleTime,ET,DL1_Miss_Rate,DL1_MISSES,DL1_ACCESSES
0,0,intel,192773,119390,0.619329,0.628193,0.000075,0.127813,11096,86814
1,1,nano,378820,946526,2.498617,1.158975,0.001097,0.119869,41235,344000
2,2,intel,0,0,nan,inf,0.000636,nan,0,0
3,3,intel,375031,182010,0.485320,0.373606,0.000068,0.195028,41383,212190
4,4,nano,739297,1951258,2.639343,0.940931,0.001836,0.110242,74751,678064
5,5,intel,0,0,nan,inf,0.000405,nan,0,0
6,6,nano,739284,1920094,2.597235,1.066094,0.002047,0.248777,176808,710710
